# Chapter 15: Dividends and Fee Management

In [4]:
# Get dividend from custodian as CSV
import yfinance as yf
import pandas as pd

newDividends = pd.read_csv('./Data/Dividends.csv')
print(newDividends)

     Account Symbol    Amount     Units
0  123456789    TLT  0.000000  0.000487
1  123456789    IEI  0.000000  0.000360
2  123456789    VTI  0.169971  0.000000
3  987654321    IEI  0.000000  0.002257
4  987654321    VTI  0.193737  0.000000


In [5]:
class Allocation:
  def __init__(self, ticker, percentage):
    self.ticker = ticker
    self.percentage = percentage
    self.units = 0.0

class Portfolio:

  def __init__(self, tickerString: str, expectedReturn: float, portfolioName: str, riskBucket: int):

    self.name = portfolioName
    self.riskBucket = riskBucket
    self.expectedReturn = expectedReturn
    self.allocations = []

    from pypfopt.efficient_frontier import EfficientFrontier
    from pypfopt import risk_models
    from pypfopt import expected_returns

    df = self.__getDailyPrices(tickerString, "20y")

    mu = expected_returns.mean_historical_return(df)
    S = risk_models.sample_cov(df)

    ef = EfficientFrontier(mu, S)

    ef.efficient_return(expectedReturn)
    self.expectedRisk = ef.portfolio_performance()[1]
    portfolioWeights = ef.clean_weights()

    for key, value in portfolioWeights.items():
      newAllocation = Allocation(key, value)
      self.allocations.append(newAllocation)

  def __getDailyPrices(self, tickerStringList, period):
    data = yf.download(tickerStringList, group_by="Ticker", period=period)
    data = data.iloc[:, data.columns.get_level_values(1)=="Close"]
    data = data.dropna()
    data.columns = data.columns.droplevel(1)
    return data

  def printPortfolio(self):
    print("Portfolio Name: " + self.name)
    print("Risk Bucket: " + str(self.riskBucket))
    print("Expected Return: " + str(self.expectedReturn))
    print("Expected Risk: " + str(self.expectedRisk))
    print("Allocations: ")
    for allocation in self.allocations:
      print("Ticker: " + allocation.ticker + ", Percentage: " + str(allocation.percentage))

  @staticmethod
  def getPortfolioMapping(riskToleranceScore, riskCapacityScore):
    allocationLookupTable=pd.read_csv('./Data/Risk Mapping Lookup.csv')
    matchTol = (allocationLookupTable['Tolerance_min'] <= riskToleranceScore) & (allocationLookupTable['Tolerance_max'] >= riskToleranceScore)
    matchCap = (allocationLookupTable['Capacity_min'] <= riskCapacityScore) & (allocationLookupTable['Capacity_max'] >= riskCapacityScore)
    portfolioID = allocationLookupTable['Portfolio'][(matchTol & matchCap)]
    return portfolioID.values[0]

class Goal:
  def __init__(self, name: str, targetYear: int, targetValue: float, portfolio: Portfolio=None, initialContribution: float=0, monthlyContribution: float=0, priority: str=""):
    self.name = name
    self.targetYear = targetYear
    self.targetValue = targetValue
    self.initialContribution = initialContribution
    self.monthlyContribution = monthlyContribution
    if not (priority == "") and not (priority in ["Dreams", "Wishes", "Wants", "Needs"]):
            raise ValueError("Wrong value set for Priority.")
    self.priority = priority
    self.portfolio = portfolio

  def getGoalProbabilities(self):
    if (self.priority == ""):
            raise ValueError("No value set for Priority.")
    lookupTable=pd.read_csv("./Data/Goal Probability Table.csv")
    match = (lookupTable["Realize"] == self.priority)
    minProb = lookupTable["MinP"][(match)]
    maxProb = lookupTable["MaxP"][(match)]
    return minProb.values[0], maxProb.values[0]

class AccountType():
  def __init__(self, value: str):
    if not value in("Taxable", "Roth IRA", "Traditional IRA"):
      raise ValueError("Allowed types: Taxable, Roth IRA, Traditional IRA")
    self.value = value
  def __eq__(self, other):
      return self.value == other.value

class AccountStatus():
  def __init__(self, value: str):
    if not value in("PENDING", "IN_REVIEW", "APPROVED", "REJECTED", "SUSPENDED"):
      raise ValueError("Allowed statuses: PENDING, IN_REVIEW, APPROVED, REJECTED, SUSPENDED")
    self.value = value
  def __eq__(self, other):
      return self.value == other.value

class Account():
  def __init__(self, number: str, accountType: AccountType, accountStatus: AccountStatus, cashBalance: float=0.0):
    self.goals = []
    self.number = number
    self.cashBalance = cashBalance
    self.accountType = accountType
    self.accountStatus = accountStatus

In [12]:
myPortfolio = Portfolio("VTI TLT IEI GLD DBC", expectedReturn = 0.05, portfolioName = "Moderate", riskBucket = 3)
myGoal = Goal(name="Vacation", targetYear=2027, targetValue=10000, priority="Dreams", portfolio=myPortfolio)
myAccount=Account(number="123456789", accountType="Taxable", accountStatus=AccountStatus("APPROVED"), cashBalance=11.0)
myAccount.goals.append(myGoal)

# Manual update using Chapter 12 outputs:
myPortfolio.allocations[0].units = 0.0
myPortfolio.allocations[1].units = 0.02213974051911262
myPortfolio.allocations[2].units = 0.02171626612838836
myPortfolio.allocations[3].units = 0.0163863407640173
myPortfolio.allocations[4].units = 0.009743440233

[*********************100%***********************]  5 of 5 completed


In [13]:
myPortfolio2 = Portfolio("VTI TLT IEI GLD DBC", expectedReturn = 0.03, portfolioName = "Conservative", riskBucket = 2)
myGoal2 = Goal(name="Car", targetYear=2025, targetValue=5000, priority="Dreams", portfolio=myPortfolio2)
myAccount2=Account(number="987654321", accountType="Taxable", accountStatus=AccountStatus("APPROVED"), cashBalance=21.0)
myAccount2.goals.append(myGoal2)

# Manual update using Chapter 12 outputs:
myPortfolio2.allocations[0].units = 0.019070282760887385
myPortfolio2.allocations[1].units = 0.0
myPortfolio2.allocations[2].units = 0.10911027337161164
myPortfolio2.allocations[3].units = 0.019273081685982695
myPortfolio2.allocations[4].units = 0.0

[*********************100%***********************]  5 of 5 completed


In [14]:
accounts = [myAccount, myAccount2]

In [15]:
for account in accounts:
  print("Account: " + str(account.number))
  print("Cash: " + str(account.cashBalance))
  for goal in account.goals:
    print("Portfolio: " + goal.portfolio.name)
    for allocation in goal.portfolio.allocations:
      print(allocation.ticker + ", units: " + str(allocation.units))
    print("\n")

Account: 123456789
Cash: 11.0
Portfolio: Moderate
GLD, units: 0.0
DBC, units: 0.02213974051911262
TLT, units: 0.02171626612838836
IEI, units: 0.0163863407640173
VTI, units: 0.009743440233


Account: 987654321
Cash: 21.0
Portfolio: Conservative
TLT, units: 0.019070282760887385
DBC, units: 0.0
GLD, units: 0.10911027337161164
IEI, units: 0.019273081685982695
VTI, units: 0.0




In [16]:
accounts

[<__main__.Account at 0x16a132bf0>, <__main__.Account at 0x16a2fd450>]

In [17]:
# Split/allocate back to account.goal(s).portfolio
for index, row in newDividends.iterrows():
  accountNo = row['Account']
  
  account = next((account for account in accounts if str(account.number) == str(accountNo)), None)

  if account == None:
    print("No account")
    break
  
  if row['Amount'] > 0:
    account.cashBalance += row['Amount']
  elif row['Units'] > 0:
    unitsToAllocate = row['Units']
    unitsAcrossPortfolios = 0.0

    for goal in account.goals:
      for allocation in goal.portfolio.allocations:
        if allocation.ticker == row['Symbol']:
          unitsAcrossPortfolios += allocation.units

    for goal in account.goals:
      for allocation in goal.portfolio.allocations:
        if allocation.ticker == row['Symbol']:
          allocation.units += unitsToAllocate * (allocation.units / unitsAcrossPortfolios)    

In [18]:
for account in accounts:
  print("Account: " + str(account.number))
  print("Cash: " + str(account.cashBalance))
  for goal in account.goals:
    print("Portfolio: " + goal.portfolio.name)
    for allocation in goal.portfolio.allocations:
      print(allocation.ticker + ", units: " + str(allocation.units))
    print("\n")

Account: 123456789
Cash: 11.1699714236
Portfolio: Moderate
GLD, units: 0.0
DBC, units: 0.02213974051911262
TLT, units: 0.02220343814008836
IEI, units: 0.0167458724207173
VTI, units: 0.009743440233


Account: 987654321
Cash: 21.1937368557
Portfolio: Conservative
TLT, units: 0.019070282760887385
DBC, units: 0.0
GLD, units: 0.10911027337161164
IEI, units: 0.021530080818982694
VTI, units: 0.0




In [19]:
# Create a function, standalone or account class
import pandas as pd
def allocateDividends(dividendFile: pd.DataFrame, accounts: list):

  for index, row in dividendFile.iterrows():
    accountNo = row['Account']
    
    account = next((account for account in accounts if str(account.number) == str(accountNo)), None)

    if account == None:
      print("No account")
      break
    
    if row['Amount'] > 0:
      account.cashBalance += row['Amount']
    elif row['Units'] > 0:
      unitsToAllocate = row['Units']
      unitsAcrossPortfolios = 0.0

      for goal in account.goals:
        for allocation in goal.portfolio.allocations:
          if allocation.ticker == row['Symbol']:
            unitsAcrossPortfolios += allocation.units

      for goal in account.goals:
        for allocation in goal.portfolio.allocations:
          if allocation.ticker == row['Symbol']:
            allocation.units += unitsToAllocate * (allocation.units / unitsAcrossPortfolios)    


In [20]:
# Show (print) an account level summary of new dividends with date included
spy = yf.Ticker("SPY")
spy.dividends

Date
1993-03-19 00:00:00-05:00    0.213
1993-06-18 00:00:00-04:00    0.318
1993-09-17 00:00:00-04:00    0.286
1993-12-17 00:00:00-05:00    0.317
1994-03-18 00:00:00-05:00    0.271
                             ...  
2021-12-17 00:00:00-05:00    1.633
2022-03-18 00:00:00-04:00    1.366
2022-06-17 00:00:00-04:00    1.577
2022-09-16 00:00:00-04:00    1.596
2022-12-16 00:00:00-05:00    1.781
Name: Dividends, Length: 121, dtype: float64

In [22]:
#spy.info['yield']

In [23]:
#Learn how to calculate Fees

In [24]:
accounts = [myAccount, myAccount2]

In [26]:
# Calculate AUM today
for account in accounts:
  print("Account: " + str(account.number))
  print("Cash: " + str(account.cashBalance))
  aum = 0.0
  for goal in account.goals:
    print("Portfolio: " + goal.portfolio.name)
    for allocation in goal.portfolio.allocations:
      price = float(yf.Ticker(allocation.ticker).basic_info["previous_close"])
      aum += price * allocation.units
  print("AUM: " + '${0:.2f}'.format(aum))
  print("\n")

Account: 123456789
Cash: 11.1699714236
Portfolio: Moderate
AUM: $6.86


Account: 987654321
Cash: 21.1937368557
Portfolio: Conservative
AUM: $24.24




In [27]:
tickerString = ""
for account in accounts:
  for goal in account.goals:
    for allocation in goal.portfolio.allocations:
      if not allocation.ticker in tickerString:
        tickerString += allocation.ticker + " "
print(tickerString)

GLD DBC TLT IEI VTI 


In [29]:
# Import the datetime module
from datetime import datetime, timedelta
from calendar import monthrange

# Get the current date
today = datetime.today()

# Get the first day of the current month
first_day_of_month = today.replace(day=1)

# Use the monthrange() method to get the number of days in the previous month
month = first_day_of_month.month-1
if month == 0:
    month = 12
_, num_days_in_prev_month = monthrange(first_day_of_month.year, month)

# Subtract the number of days in the previous month from the first day of the current month to get the first day of the previous month
first_day_of_prev_month = first_day_of_month - timedelta(days=num_days_in_prev_month)

# Get the last day of the previous month by adding one day to the first day of the current month, and then subtracting one day
last_day_of_prev_month = first_day_of_month - timedelta(days=1)

# Print the first and last dates of the previous month
print(f"First date of previous month: {first_day_of_prev_month:%Y-%m-%d}")
print(f"Last date of previous month: {last_day_of_prev_month:%Y-%m-%d}")


First date of previous month: 2022-12-01
Last date of previous month: 2022-12-31


In [30]:
# Get price history over period for all accounts and tickers (one month)
data = yf.download(tickerString, group_by="Ticker", start=first_day_of_prev_month, end=last_day_of_prev_month)
data = data.iloc[:, data.columns.get_level_values(1)=="Close"]
data = data.dropna()
data.columns = data.columns.droplevel(1)
print(data)

[*********************100%***********************]  5 of 5 completed
                                  GLD         VTI        DBC         IEI  \
Date                                                                       
2022-12-01 00:00:00-05:00  167.839996  204.179993  25.360001  116.430000   
2022-12-02 00:00:00-05:00  167.259995  203.990005  25.090000  116.529999   
2022-12-05 00:00:00-05:00  164.389999  199.979996  24.440001  115.720001   
2022-12-06 00:00:00-05:00  164.839996  196.979996  24.049999  116.059998   
2022-12-07 00:00:00-05:00  166.330002  196.660004  23.930000  116.720001   
2022-12-08 00:00:00-05:00  166.470001  198.360001  23.889999  116.290001   
2022-12-09 00:00:00-05:00  167.059998  196.699997  23.920000  115.940002   
2022-12-12 00:00:00-05:00  165.679993  199.529999  24.180000  115.860001   
2022-12-13 00:00:00-05:00  168.509995  201.020004  24.580000  116.540001   
2022-12-14 00:00:00-05:00  168.100006  199.899994  24.770000  116.760002   
2022-12-15 00:00:00

In [31]:
# Calculate average AUM over period
for account in accounts:
  print("Account: " + str(account.number))
  print("Cash: " + str(account.cashBalance))
  aum = 0.0
  for goal in account.goals:
    print("Portfolio: " + goal.portfolio.name)
    for allocation in goal.portfolio.allocations:
      for index, row in data.iterrows():
        price = row[allocation.ticker]
        aum += price * allocation.units
  aum = aum / len(data)
  print("Average AUM: " + '${0:.2f}'.format(aum))
  print("\n")

Account: 123456789
Cash: 11.1699714236
Portfolio: Moderate
Average AUM: $6.72


Account: 987654321
Cash: 21.1937368557
Portfolio: Conservative
Average AUM: $22.75




In [32]:
# Calculate fee
feeRate = 0.001 # 10bps

for account in accounts:
  print("Account: " + str(account.number))
  print("Cash: " + str(account.cashBalance))
  aum = 0.0
  for goal in account.goals:
    print("Portfolio: " + goal.portfolio.name)
    for allocation in goal.portfolio.allocations:
      for index, row in data.iterrows():
        price = row[allocation.ticker]
        aum += price * allocation.units
  aum = aum / len(data)
  fee = aum * feeRate * (num_days_in_prev_month / 365)
  print("Average AUM: " + '${0:.2f}'.format(aum))
  print("Fee: $" + str(fee))
  print("\n")

Account: 123456789
Cash: 11.1699714236
Portfolio: Moderate
Average AUM: $6.72
Fee: $0.0005706570443678678


Account: 987654321
Cash: 21.1937368557
Portfolio: Conservative
Average AUM: $22.75
Fee: $0.001931956557080065




In [33]:
# May need to create sell order(s) if cash < AUM

In [34]:
# Generate broker instructions for fees (CSV?)
feeRate = 0.001 # 10bps
feeTable = pd.DataFrame(columns=['Account','FeeAmount'])

for account in accounts:
  aum = 0.0
  for goal in account.goals:
    for allocation in goal.portfolio.allocations:
      for index, row in data.iterrows():
        price = row[allocation.ticker]
        aum += price * allocation.units
  aum = aum / len(data)
  fee = aum * feeRate * (num_days_in_prev_month / 365)
  new_row = {'Account':account.number, 'FeeAmount':fee}
  feeTable = feeTable.append(new_row, ignore_index=True)

print(feeTable)

     Account  FeeAmount
0  123456789   0.000571
1  987654321   0.001932


In [35]:
pd.set_option("display.precision", 8)

In [36]:
# Generate broker instructions for fees (CSV?)
feeRate = 0.001 # 10bps
feeTable = pd.DataFrame(columns=['Account','FeeAmount'])

for account in accounts:
  aum = 0.0
  for goal in account.goals:
    for allocation in goal.portfolio.allocations:
      for index, row in data.iterrows():
        price = row[allocation.ticker]
        aum += price * allocation.units
  aum = aum / len(data)
  fee = aum * feeRate * (num_days_in_prev_month / 365)
  new_row = {'Account':account.number, 'FeeAmount':fee}
  feeTable = feeTable.append(new_row, ignore_index=True)

print(feeTable)

     Account   FeeAmount
0  123456789  0.00057066
1  987654321  0.00193196


In [39]:
# Create function
def generateFeeInstructions(accounts: list) -> pd.DataFrame:
  tickerString = ""
  for account in accounts:
    for goal in account.goals:
      for allocation in goal.portfolio.allocations:
        if not allocation.ticker in tickerString:
          tickerString += allocation.ticker + " "
  
  from datetime import datetime, timedelta
  from calendar import monthrange
  today = datetime.today()
  first_day_of_month = today.replace(day=1)
  month = first_day_of_month.month-1
  if month == 0:
      month = 12
  _, num_days_in_prev_month = monthrange(first_day_of_month.year, month)
  first_day_of_prev_month = first_day_of_month - timedelta(days=num_days_in_prev_month)
  last_day_of_prev_month = first_day_of_month - timedelta(days=1)

  data = yf.download(tickerString, group_by="Ticker", start=first_day_of_prev_month, end=last_day_of_prev_month)
  data = data.iloc[:, data.columns.get_level_values(1)=="Close"]
  data = data.dropna()
  data.columns = data.columns.droplevel(1)

  feeRate = 0.001 # 10bps
  feeTable = pd.DataFrame(columns=['Account','FeeAmount'])

  for account in accounts:
    aum = 0.0
    for goal in account.goals:
      for allocation in goal.portfolio.allocations:
        for index, row in data.iterrows():
          price = row[allocation.ticker]
          aum += price * allocation.units
    aum = aum / len(data)
    fee = aum * feeRate * (num_days_in_prev_month / 365)
    new_row = {'Account':account.number, 'FeeAmount':fee}
    feeTable = feeTable.append(new_row, ignore_index=True)

  return feeTable

In [40]:
# Save to CSV
feeFile = generateFeeInstructions(accounts)

[*********************100%***********************]  5 of 5 completed


In [41]:
feeFile.to_csv('./Data/Fees.csv')